# Adversal validation
1. Isna by axis=0\
\
First import the essentials

In [1]:
import os
import sys
# import keras
# import tensorflow as tf
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
    sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/packages')
    !pip install git+https://github.com/keras-team/keras-tuner.git
    !pip install autokeras
    config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 2} )
    sess = tf.Session(config=config)
    keras.backend.set_session(sess)
else:
    path_fill = 'data/2022/filled_trian.csv'
    path_raw_train='data/2022/train.csv'
    path_A='data/2022/test_A榜.csv'
    path_B='data/2022/test_B榜.csv'

In [2]:
import pandas as pd
import useful_functions as uf
# import autokeras as ak
df_train = pd.read_csv(path_raw_train).head(500)
df_train_filled = pd.read_csv(path_fill,index_col=0).head(500)
df_test_A = pd.read_csv(path_A).head(500)
df_test_B = pd.read_csv(path_B).head(500)

In [3]:
import numpy as np
import sklearn_pandas as sp
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

2. Drop too few columns, optional drop index for train data

In [4]:
def drop_too_few_variable(dataframe: pd.DataFrame, threshold=0.75, whether_index = False):
    total_rows = len(df_test_A.index)
    mask_drop = (dataframe.count() / total_rows ) > threshold
    dataframe = dataframe.loc[:, mask_drop]
    if whether_index:
        drop_index = dataframe.isna().sum(axis=1)>=len(dataframe.columns)*threshold
        drop_index = [ind for ind in drop_index.index if drop_index[ind]==True]
        dataframe.drop(index = drop_index)
        return dataframe.drop(index = drop_index)
    else:
        return dataframe
df_train = drop_too_few_variable(df_train,0.75,True)
portfolio:list = [df_test_A,df_test_B]
for i in range(len(portfolio)):
    portfolio[i] = drop_too_few_variable(portfolio[i])

3. step 1, minus 2

In [5]:
s_col = uf.dynamic_string_col(df_train)
df_train,string_col = uf.step1_data_processing(df_train,s_col)

# Need try iqr first or fillna first
\
Slip into two, nu and not nu
\
4.  Fillna, try random_forest, KNN, mode
## 3.2 数据集中缺失值从少到多进行排序
从缺失值最少的开始填，所以首先需要一个排序，即数据集中缺失值从少到多的一个顺序
# 3.3构建新特征矩阵和新标签

### Fill the string col first, fill with mode, then fill with iteraimpute

In [6]:
s_col = uf.dynamic_string_col(df_train)
df_train_str = df_train.loc[:,s_col]
impMode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
impMode.fit(df_train_str)
df_train_str = \
    pd.DataFrame(impMode.transform(df_train_str),columns=s_col,index=df_train_str.index)

In [17]:


df_nu = uf.get_numerical_df(df_train)
sor_by_nan = df_nu.isna().sum().sort_values(ascending=True)
# star with the least missing number
con = sor_by_nan>0 # Filter out those with no missing values
sor_by_nan =sor_by_nan[con].index # index
i=0    # loop over all columns with nan
to_be_fillc:str = sor_by_nan[i]
x_tobe_0 = sor_by_nan[sor_by_nan!=to_be_fillc] # No train data, thus fill 0
# columns names that are going to be filled woth zero as training data
#------only x train, y_train(label), x_test, only three----------
y_train =df_nu[to_be_fillc][df_nu[to_be_fillc].notna()]
x_train_nofill = df_nu.loc[y_train.index,x_tobe_0]

x_test_index = df_nu[df_nu[to_be_fillc].isna()].index
x_test_nofill =df_nu.loc[x_test_index,x_tobe_0]

#---------------- FILL IN ALL ZERO x train --------------------------------
imp0 = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
x_train=imp0.fit_transform(x_train_nofill)
x_tested=imp0.transform(x_test_nofill)

rfc = RandomForestRegressor()
rfc.fit(x_train,y_train)
y_pred = pd.Series(\
    rfc.predict(x_tested),\
    index=x_test_index\
    )


complete_y_pred = pd.concat([y_train,y_pred])
complete_y_pred.name = to_be_fillc
df_nu.loc[:,to_be_fillc]

0      191980910.7
1       74162774.4
2       14013308.2
3        7359191.5
4              NaN
          ...     
495    134626666.7
496      9921125.1
497      3882542.1
498    249270991.4
499     10519441.5
Name: LAST_12_MON_MON_AVG_TRX_AMT_NAV, Length: 498, dtype: float64

In [22]:
df_train.loc[complete_y_pred.index,to_be_fillc] = complete_y_pred
# df_nu.loc[:,to_be_fillc]
df_train.loc[:,to_be_fillc]

0      1.919809e+08
1      7.416277e+07
2      1.401331e+07
3      7.359192e+06
4      3.201943e+06
           ...     
495    1.346267e+08
496    9.921125e+06
497    3.882542e+06
498    2.492710e+08
499    1.051944e+07
Name: LAST_12_MON_MON_AVG_TRX_AMT_NAV, Length: 498, dtype: float64